In [ ]:
import findspark

In [ ]:
findspark.init()

In [ ]:
findspark.find()

In [ ]:
from pyspark.sql import SparkSession

### Spark Session object

In [ ]:
spark = SparkSession.builder.appName("Anaconda python").master("local[5]").getOrCreate()

In [ ]:
spark

### Create Spark Context from Spark session object

In [ ]:
sc = spark.sparkContext

### Create rdd from memory

In [ ]:
numbers_rdd = sc.parallelize(range(1,101), 4)

In [ ]:
numbers_rdd

## Map partitions with Index

In [ ]:
def print_elements_in_partition(index, elements):
    for element in elements:
        yield f"partition: {index} has element: {element}"

In [ ]:
numbers_rdd.mapPartitionsWithIndex(print_elements_in_partition).collect()

### Map transfromation

In [ ]:
numbers_rdd.map(lambda x: x*x).collect()

### Filter transformation

In [ ]:
numbers_rdd.filter(lambda x : x%2 ==0).collect()

### Get Number of partitions of a rdd

In [ ]:
numbers_rdd.getNumPartitions()

### Count Action

In [ ]:
numbers_rdd.count()

### flatMap transformation

In [ ]:
sentences = ["This is a sentence1.", "This is sentence2.", "This is sentence3"]

In [ ]:
sentence_rdd = sc.parallelize(sentences)

In [ ]:
sentence_rdd.collect()

In [ ]:
sentence_rdd.flatMap(lambda x:x.split()).collect()

#### Get the number of rdd partitions

In [ ]:
rdd = sc.textFile("/Users/avinashs/Downloads/pyspark/datasets/Baby_Names__Beginning_2007.csv")

In [ ]:
rdd

In [ ]:
rdd.getNumPartitions()

In [ ]:
rdd.collect()

### Count the number of elements in a rdd

In [ ]:
rdd.count()

### Mappartitions index transformation

In [ ]:
def count_baby_names_partitions_wise(index, iterator):
    count_records = 0
    for row in iterator:
        count_records += 1
    yield f"Parition {index} has {count_records} records"

In [ ]:
rdd.mapPartitionsWithIndex(count_baby_names_partitions_wise).collect()

In [ ]:
def skip_header(index, iterator):
    number = 0
    for row in iterator:
        if index == 0 and number == 0:
            number += 1
            continue
        yield row
            
    
            

In [ ]:
data_rdd = rdd.mapPartitionsWithIndex(skip_header)

In [ ]:
year_rdd = data_rdd.map(lambda x: (x.split(",")[0], int(x.split(",")[-1])))

### reducebykey and groupbykey transformations

In [ ]:
year_rdd.reduceByKey(lambda x, y : x+y).collect()

In [ ]:
year_rdd.groupByKey().mapValues(sum).collect()

In [ ]:
year_rdd.getNumPartitions()

### Increase the number of partition

In [ ]:
year_rdd = year_rdd.repartition(4)

In [ ]:
year_rdd.getNumPartitions()

### Decrease the number of partition

In [ ]:
year_rdd.coalesce(3).getNumPartitions()

### Write rdd to a text file

In [ ]:
year_rdd.saveAsTextFile("years.txt")

In [ ]:
rdd_repartition = rdd.repartition(4)

In [ ]:
rdd_repartition.collect()

In [ ]:
file_rdd = sc.textFile("/Users/avinashs/PycharmProjects/introtoPython/spark/datasets/ebook.txt", 4)

In [ ]:
file_rdd.collect()

In [ ]:
file_rdd.flatMap(lambda x: x.split(" ")).map(lambda x : (x,1)).reduceByKey(lambda x,y: x+y).collect()

In [ ]:
filename = "/Users/avinashs/PycharmProjects/introtoPython/spark/datasets/weather.csv"
file1_rdd = sc.textFile(filename)

### take action

In [ ]:
file1_rdd.take(5)

In [ ]:
file1_rdd.first()

In [ ]:
file1_rdd.map(lambda x: (x.split(",")[0], int(x.split(",")[-1]))).map(lambda x: (int(x[0].split("-")[0]), x[1])).reduceByKey(lambda x, y: max(x,y)).sortByKey().collect()

### Accumulators

In [ ]:
accum = sc.accumulator(0)

In [ ]:
file_rdd = sc.textFile("/Users/avinashs/PycharmProjects/introtoPython/spark/datasets/ebook.txt")

In [ ]:
file_rdd.getNumPartitions()

In [ ]:
def count_blank_lines_in_partition(iterator):
    count_blank_lines = 0
    for row in iterator:
        if len(row.strip()) == 0:
            count_blank_lines += 1

    accum.add(count_blank_lines)

In [ ]:
file_rdd.foreachPartition(count_blank_lines_in_partition)

In [ ]:
accum.value

### read json file in python

In [ ]:
import json
from pprint import pprint

filename = "/Users/avinashs/PycharmProjects/introtoPython/spark/datasets/people.json"
texts = []
for line in open(filename, 'r'):
    texts.append(json.loads(line))

In [ ]:
json_rdd = sc.textFile(filename)

In [ ]:
def convert_to_csv(record):
    return tuple(json.loads(record).values())

In [ ]:
json_rdd.map(convert_to_csv).saveAsTextFile("json_csv")

### Store the rdd in memory or in disk

In [ ]:
rdd = sc.textFile("/Users/avinashs/PycharmProjects/introtoPython/spark/datasets/Baby_Names__Beginning_2007.csv")

In [ ]:
rdd.take(5)

In [ ]:
first_rdd = rdd.map(lambda x : (x.split(",")[0], x.split(",")[1]))

In [ ]:
second_rdd = rdd.map(lambda x: (x.split(",")[1], x.split(",")[-2]))

In [ ]:
rdd.persist()

In [ ]:
first_rdd.collect()

In [ ]:
second_rdd.collect()

In [ ]:
rdd = rdd.unpersist()

In [ ]:
rdd = rdd.unpersist(blocking=False)

In [ ]:
rdd.collect()

In [ ]:
help(rdd.cache)

In [ ]:
help(rdd.persist)

In [ ]:
rdd.is_cached

In [ ]:
rdd.unpersist()

### To see all the spark configuration variables

In [ ]:
sc._conf.getAll()

### foreachPartition action

In [ ]:
rdd.foreachPartition(print)

### stop the sparksession

In [ ]:
spark.stop()